## ECE176 Final Project - Audio Preprocessing

In [ ]:
import os
import librosa
import numpy as np

#### Convert to MelSpec

In [ ]:
def process_mp3_file(mp3_path, target_folder, segment_len_ms=15000, overlap_ms=5000, sr=44100):
    y, sr = librosa.load(mp3_path, sr=sr)
    
    # Convert milliseconds to samples
    segment_len_samples = int(sr * segment_len_ms / 1000)
    overlap_samples = int(sr * overlap_ms / 1000)
    step_size_samples = segment_len_samples - overlap_samples
    
    # Calculate the number of segments
    num_segments = 1 + (len(y) - segment_len_samples) // step_size_samples
    
    for i in range(num_segments):
        start_samples = i * step_size_samples
        end_samples = start_samples + segment_len_samples
        
        # Ensure we don't go beyond the file length
        if end_samples > len(y):
            break
        
        segment = y[start_samples:end_samples]
        
        # Convert to Mel spectrogram
        S = librosa.feature.melspectrogram(y=segment, sr=44100, n_mels=256)       
        
        # Save as a numpy array file
        base_name = os.path.splitext(os.path.basename(mp3_path))[0]
        target_file_name = f"{base_name}_segment_{i+1}.npy"
        target_file_path = os.path.join(target_folder, target_file_name)
        
        np.save(target_file_path, S)

In [ ]:
#convert to melspec
if __name__ == "__main__":
    audio_folder = 'C:/Eric/UCSD/ECE/ECE176_Final/dataset/'
    spec_folder = 'C:/Eric/UCSD/ECE/ECE176_Final/dataset/'

    # Ensure target folder exists
    if not os.path.exists(audio_folder):
        os.makedirs(audio_folder)
    elif not os.path.exists(spec_folder):
        os.makedirs(spec_folder)
        
    for file_name in os.listdir(audio_folder):
        if file_name.endswith(".mp3"):
            mp3_path = '{}/{}'.format(audio_folder, file_name)
            process_mp3_file(mp3_path, spec_folder)
            print(f"Processed {file_name}")

In [ ]:
#random inspection
from IPython.display import Audio
import matplotlib.pyplot as plt

import random
def random_sample(directory_path):
    files = [f for f in os.listdir(directory_path) if f.endswith('.npy')]
    random_file = random.choice(files)

    mel_spec_path = os.path.join(directory_path, random_file)
    mel_spec = np.load(mel_spec_path)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Convert melspec to wave
    audio = librosa.feature.inverse.mel_to_audio(mel_spec, sr=44100, n_fft=2048, hop_length=512)

    # Plot
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, x_axis='time', y_axis='mel', sr=44100, fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel Spectrogram')
    plt.tight_layout()
    plt.show()

    print(f"Playing: {random_file}")
    print(f"MelSpec Dimensions: {mel_spec.shape}")
    return Audio(audio, rate=44100)

directory_path = 'C:/Eric/UCSD/ECE/ECE176_Final/dataset/'
audio = random_sample(directory_path)
audio

#### Data Inspection (optional)

In [ ]:
# calculate duration of the dataset
import eyed3
def get_total_duration(directory):
    total_duration = 0.0
    for filename in os.listdir(directory):
        if filename.endswith(".mp3"):
            try:
                path = os.path.join(directory, filename)
                audio = eyed3.load(path)
                if audio is not None and audio.info is not None:
                    total_duration += audio.info.time_secs
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return total_duration

# Replace 'your_directory_path' with the path to your directory containing MP3 files
directory_path = 'C:/Eric/UCSD/ECE/ECE176_Final/dataset/pre_training_data/'
total_duration_seconds = get_total_duration(directory_path)

# Convert seconds to hours, minutes, and seconds
hours, remainder = divmod(total_duration_seconds, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"Total Duration: {int(hours)} hours, {int(minutes)} minutes, and {int(seconds)} seconds")

In [ ]:
# for renaming, unifying naming convention, checking invalid values
path = 'C:/Eric/UCSD/ECE/ECE176_Final/dataset/hoyo/spectrograms/'

files = os.listdir(path)

for filename in files:
    # construct full file path
    file_path = os.path.join(path, filename)
    
    # load the numpy array
    array = np.load(file_path)
    
    # check if the array contains any NaN values
    if np.any(np.isnan(array)):
        print(f"File {filename} contains NaN values")
        # delete the file
        #os.remove(file_path)